Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep
This notebook demonstrates use the AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we use AzureML on a preloaded dataset. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook we:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [31]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from sklearn.model_selection import train_test_split

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize Workspace
Initialize a workspace object.

In [5]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136985
aml-quickstarts-136985
southcentralus
1b944a9b-fdae-4f97-aeb1-b7eea0beac53


## Create an Azure ML experiment
Let's create an experiment named "automlstep-regression" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

In [26]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automlstep-regression'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automlstep-regression,quick-starts-ws-136985,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. Here, we use the name `automl` as training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster if present.

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded...............................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [27]:
ws.datasets.keys()

KeysView({'Houses': DatasetRegistration(id='057bd665-fb78-4ffc-a8a5-1bfd9f40c301', name='Houses', version=1, description='House prices and characteristics.', tags={})})

In [28]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Houses"
description_text = "House prices and characteristics."

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        from azureml.core import Workspace, Dataset

        subscription_id = '1b944a9b-fdae-4f97-aeb1-b7eea0beac53'
        resource_group = 'aml-quickstarts-136985'
        workspace_name = 'quick-starts-ws-136985'
        workspace = Workspace(subscription_id, resource_group, workspace_name)
        dataset = Dataset.get_by_name(workspace, name='Houses')

df = dataset.to_pandas_dataframe()
df.describe()

,Transactieprijs_m2,BU_g_woz_v2018,BU_g_woz_v2016,WK_g_woz_v2019,BU_g_ink_po_v2017,Woonoppervlakte,BAGbouwjaar,maanden_sinds_jan2004,BUURT_m2_alle_objecten,Woningtype_appartement,...,Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345
count,51314.00,51314.00,51314.00,51314.00,51314.00,51314.00,51314.00,51314.00,51314.00,51314.00,...,50458.00,50774.00,50822.00,51314.00,51314.00,48512.00,51263.00,51202.00,51070.00,50961.00
mean,3545.26,272.38,220.08,303.71,35.01,101.16,1952.50,109.13,282246.78,0.46,...,-1.00,-1.00,-1.00,265.70,277.55,-1.00,-1.00,-1.00,-1.00,-1.00
std,1237.78,94.27,78.05,66.24,8.62,46.01,64.17,60.58,191316.94,0.50,...,0.00,0.00,0.00,1006.34,951.01,0.00,0.00,0.00,0.00,0.00
min,207.43,-1.00,-1.00,177.00,-1.00,11.00,1250.00,0.00,21498.00,0.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
25%,2625.00,212.00,164.00,250.00,29.10,74.00,1926.00,52.00,182797.00,0.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
50%,3294.12,265.00,214.00,328.00,33.90,93.00,1961.00,119.00,245066.00,0.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
75%,4200.00,326.00,262.00,354.00,41.10,120.00,1997.00,162.00,326001.00,1.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
max,23890.91,771.00,647.00,403.00,63.80,1145.00,2020.00,203.00,1649683.00,1.00,...,-1.00,-1.00,-1.00,5326.22,4895.16,-1.00,-1.00,-1.00,-1.00,-1.00


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [29]:
dataset.take(5).to_pandas_dataframe()

,Transactieprijs_m2,BU_g_woz_v2018,BU_g_woz_v2016,WK_g_woz_v2019,BU_g_ink_po_v2017,Woonoppervlakte,BAGbouwjaar,maanden_sinds_jan2004,BUURT_m2_alle_objecten,Woningtype_appartement,...,Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345
0,2700.00,156,137,219,28.10,40,1962,1,290265,1,...,-1,-1,-1,-1.00,-1.00,-1,-1,-1,-1,-1
1,1725.00,149,125,219,23.10,80,1956,1,413897,1,...,-1,-1,-1,-1.00,-1.00,-1,-1,-1,-1,-1
2,2350.00,189,151,247,28.10,60,1979,1,184252,1,...,-1,-1,-1,-1.00,-1.00,-1,-1,-1,-1,-1
3,2875.00,213,167,250,26.90,48,1919,1,116432,1,...,-1,-1,-1,-1.00,-1.00,-1,-1,-1,-1,-1
4,2057.14,156,137,219,28.10,70,1962,1,290265,1,...,-1,-1,-1,-1.00,-1.00,-1,-1,-1,-1,-1


### Train - Test split

Now that we have the data, we do not need to create a train and test set as we've done in the HyperDrive example. Based on Microsoft documentation, if you do not explicitly specify either validation_data or n_cross_validation parameters, AutoML applies default techniques based on the number of rows in the dataset (if >20k rows, it will make a train-validation split where 10% is reserved for the validation set and where metrics returned are based on the validation set; if less than 20k rows, it will apply 10-fold cross-validation in case of < 1,000 rows, otherwise 3-fold cross-validation). In the HyperDrive example, we've chosen for a validation/test set of 20%. Therefore, we modify the parameter to use a 20% in the AutoML settings as well (so that results can be compared).

## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `Transactieprijs_m2` (the actual price sold) in this example.

In [107]:
automl_settings = {
    "experiment_timeout_minutes": 50,
    "max_concurrent_iterations": 5,
    "primary_metric": 'normalized_mean_absolute_error'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="Transactieprijs_m2",   
                             path = project_folder,
                             validation_size = 0.2,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [108]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [109]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [110]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [111]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [a6634a48][cbbbcdd8-0e0f-4573-90ef-8a32fae3e9c5], (This step will run and generate new outputs)
Submitted PipelineRun 38a103df-588a-42be-8d22-d009c501df67
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-regression/runs/38a103df-588a-42be-8d22-d009c501df67?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-136985/workspaces/quick-starts-ws-136985


In [112]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [113]:
pipeline_run.wait_for_completion()

PipelineRunId: 38a103df-588a-42be-8d22-d009c501df67
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-regression/runs/38a103df-588a-42be-8d22-d009c501df67?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-136985/workspaces/quick-starts-ws-136985
PipelineRun Status: Running


StepRunId: 36901ecd-1b13-4191-9f12-6c7bb22136b6
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-regression/runs/36901ecd-1b13-4191-9f12-6c7bb22136b6?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-136985/workspaces/quick-starts-ws-136985
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '36901ecd-1b13-4191-9f12-6c7bb22136b6', 'target': 'automl', 'status': 'Completed', 'startTimeUtc': '2021-02-02T21:18:34.584373Z', 'endTimeUtc': '2021-02-02T21:43

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [114]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/36901ecd-1b13-4191-9f12-6c7bb22136b6/metrics_data, 1 files out of an estimated total of 1


In [115]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,36901ecd-1b13-4191-9f12-6c7bb22136b6_14,36901ecd-1b13-4191-9f12-6c7bb22136b6_12,36901ecd-1b13-4191-9f12-6c7bb22136b6_24,36901ecd-1b13-4191-9f12-6c7bb22136b6_7,36901ecd-1b13-4191-9f12-6c7bb22136b6_20,36901ecd-1b13-4191-9f12-6c7bb22136b6_19,36901ecd-1b13-4191-9f12-6c7bb22136b6_22,36901ecd-1b13-4191-9f12-6c7bb22136b6_32,36901ecd-1b13-4191-9f12-6c7bb22136b6_41,36901ecd-1b13-4191-9f12-6c7bb22136b6_3,...,36901ecd-1b13-4191-9f12-6c7bb22136b6_13,36901ecd-1b13-4191-9f12-6c7bb22136b6_17,36901ecd-1b13-4191-9f12-6c7bb22136b6_8,36901ecd-1b13-4191-9f12-6c7bb22136b6_5,36901ecd-1b13-4191-9f12-6c7bb22136b6_23,36901ecd-1b13-4191-9f12-6c7bb22136b6_9,36901ecd-1b13-4191-9f12-6c7bb22136b6_10,36901ecd-1b13-4191-9f12-6c7bb22136b6_29,36901ecd-1b13-4191-9f12-6c7bb22136b6_34,36901ecd-1b13-4191-9f12-6c7bb22136b6_26
mean_absolute_percentage_error,[29.069116874552172],[29.154270297321826],[29.0836077363343],[29.098339642488536],[29.099902796200727],[29.122206634772557],[29.079785783400716],[29.086556669980258],[29.391642770561745],[29.047597162114865],...,[29.11903209494596],[29.149606796994675],[29.07969772403138],[29.123885949847985],[29.09381414808151],[29.135397130881497],[29.134754764378144],[29.151822217842415],[29.08980431602144],[29.098339642488387]
mean_absolute_error,[964.0743473796502],[965.2976996475728],[963.9461888464718],[964.8298940349446],[964.3284637509664],[964.7835506953584],[963.9541799271877],[964.0161469949475],[967.9559231193065],[963.4768046773946],...,[964.4709791509897],[965.2912105026135],[963.9417473680642],[964.5988620200591],[964.7552763854126],[965.2417220867754],[964.9318525054326],[965.083880757247],[964.2724224190717],[964.8298940349426]
root_mean_squared_error,[1243.3101964576354],[1243.5405000153091],[1243.1641600628868],[1244.2742455525538],[1243.4306439720676],[1243.7959327654476],[1242.973472463556],[1242.9222182996575],[1242.750770880167],[1242.9861253980173],...,[1242.9442979237497],[1244.060234470112],[1242.9852021136078],[1242.9637498443496],[1244.5324910765246],[1243.582611665312],[1243.5718030333023],[1243.2978143397825],[1243.1326198432428],[1244.2742455525538]
normalized_root_mean_squared_log_error,[0.07002158517573531],[0.0700925750778182],[0.070029950850551],[0.07008285242510269],[0.07006188629208374],[0.07008806400764911],[0.07001434644088893],[0.07002086314762343],[0.07024054339609516],[0.06999614090396836],...,[0.07004270911434472],[0.07012072030397436],[0.0700153759429892],[0.07004521343886057],[0.0700990494026277],[0.07007709222803672],[0.07008485721167391],[0.0700831262721542],[0.07002588947694277],[0.07008285242510259]
normalized_root_mean_squared_error,[0.052496937493413075],[0.052506661721128854],[0.05249077132224459],[0.0525376430431791],[0.05250202321188816],[0.05251744699189716],[0.0524827198198831],[0.0524805556884812],[0.05247331657430837],[0.05248325407136066],...,[0.0524814879680114],[0.0525286067411767],[0.052483215087039556],[0.052482309296641036],[0.05254854707916985],[0.052508439823379],[0.05250798344484964],[0.05249641467676698],[0.05248943958303429],[0.0525376430431791]
spearman_correlation,[-0.002678596854182151],[-0.012107049178510939],[-0.0006423361010152634],[-0.014941742757138372],[-0.001289029015800639],[-0.007471196274018466],[0.0024499173485633746],[0.014227273126899516],[0.019308981067791674],[0.012906189365664266],...,[-0.014299789415999865],[-0.014132861532979121],[0.00020518790366751377],[-0.005643437600797988],[-0.002586949819980439],[-0.014878273194886563],[-0.006453042165050511],[-0.0011782371929175528],[-0.01994396550860715],[-0.014941742757138372]
root_mean_squared_log_error,[0.33202126285712236],[0.33235787558704716],[0.3320609303669336],[0.3323117736968254],[0.33221235861011017],[0.33233648545126804],[0.3319869389571542],[0.33201783921732725],[0.33305949677676827],[0.3319006137855062],...,[0.3321214262104085],[0.3324913317707127],[0.33199182054597554],[0.33213330096283333],[0.3323885748848994],[0.3322844605776015],[0.33232127979080295],[0.33231307219704703],[0.33204

Here we can see that the best model has an mean_absolute_error (MAE) on validation data of: 

### Retrieve the Best Model

In [116]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/36901ecd-1b13-4191-9f12-6c7bb22136b6/model_data, 1 files out of an estimated total of 1


In [117]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [118]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('StandardScalerWrapper',
  <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper at 0x7fcc80428f28>),
 ('DecisionTreeRegressor',
  DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                        max_features=0.9, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=0.004312849662788328,
                        min_samples_split=0.018261584682702607,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best'))]

### Test the Model
#### Load Test Data
Test data is already prepared.

#### Testing Our Best Fitted Model

We will evaluate based on the mean absolute error (MAE). This is the same metric we used in the HyperDrive example. The MAE is a common evaluation measure within real estate and specifically in predicting prices. The MAE of the best HyperDrive model was 525.5451960245264. 

In [119]:
from sklearn.metrics import mean_absolute_error
df_test = dataset.to_pandas_dataframe()
y_test = df_test['Transactieprijs_m2']
x_test = df_test.drop(['Transactieprijs_m2'], axis=1)
preds = best_model.predict(x_test) 
mae = mean_absolute_error(y_test, preds)

print('The Mean Absolute Error (MAE) of the best AutoML model is on the train-dataset: ', mae)

The Mean Absolute Error (MAE) of the best AutoML model is:  976.1376070502165


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [22]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Houses_train", description="Training house price prediction pipeline", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
Bankmarketing Train,460e051f-9b77-4bf4-8be5-593f24ee6f35,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [23]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [24]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [25]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  380cd90a-2758-4734-8fbd-d639a54c0a51


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [29]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [ ]:
import requests
import json

# URL for the web service
scoring_uri = '<your web service URI>'
# If the service is authenticated, set the key or token
key = '<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [
                "BU_g_woz_v2018": 156,
                "BU_g_woz_v2016": 137,
                "WK_g_woz_v2019": 219,
                "BU_g_ink_po_v2017": 28.1,
                "Woonoppervlakte": 40,
                "BAGbouwjaar": 1962,
                "maanden_sinds_jan2004": 1,
                "BUURT_m2_alle_objecten": 290265,
                "Woningtype_appartement": 1,
                "BAG_perceeloppervlakte": 3294,
                "Woningtype_vrijstaand": 0,
                "Woningtype_tussenwoning": 0,
                "BU_ste_oad_v2018": 3475,
                "WK_ste_oad_v2018": 3460,
                "Hoofdweg_Distance": 4589,
                "hoogte": 17.17,
                "Gemeentehuis_Distance": 2158,
                "Trein_Distance": 1308,
                "Park_Distance": 1196,
                "WIJK_aantal_objecten": 21470,
                "Autosnelweg_Distance": 1468,
                "BUURT_median_bouwjaar_appartementen": 1968,
                "Koop_historisch_2020_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2019_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2018_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2017_med_transactieprijsm2_PC_123456": 2553.3126,
                "Koop_historisch_2016_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2015_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2014_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2013_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2012_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2011_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2010_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_PC_12345": 3225.4902,
                "Koop_historisch_2019_med_transactieprijsm2_PC_12345": 3256.1728,
                "Koop_historisch_2018_med_transactieprijsm2_PC_12345": 3132.3529,
                "Koop_historisch_2017_med_transactieprijsm2_PC_12345": 2628.5714,
                "Koop_historisch_2016_med_transactieprijsm2_PC_12345": 1957.1429,
                "Koop_historisch_2015_med_transactieprijsm2_PC_12345": 1816.5631,
                "Koop_historisch_2014_med_transactieprijsm2_PC_12345": 1695.0704,
                "Koop_historisch_2013_med_transactieprijsm2_PC_12345": 1739.0873,
                "Koop_historisch_2012_med_transactieprijsm2_PC_12345": 1620,
                "Koop_historisch_2011_med_transactieprijsm2_PC_12345": 2114.2857,
                "Koop_historisch_2010_med_transactieprijsm2_PC_12345": 1966.6667,
                "Koop_historisch_2020_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2019_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2018_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2017_med_transactieprijsm2_appartementen_PC_123456": 2553.3126,
                "Koop_historisch_2016_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2015_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2014_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2013_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2012_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_appartementen_PC_12345": 3225.4902,
                "Koop_historisch_2019_med_transactieprijsm2_appartementen_PC_12345": 3256.1728,
                "Koop_historisch_2018_med_transactieprijsm2_appartementen_PC_12345": 3132.3529,
                "Koop_historisch_2017_med_transactieprijsm2_appartementen_PC_12345": 2628.5714,
                "Koop_historisch_2016_med_transactieprijsm2_appartementen_PC_12345": 1957.1429,
                "Koop_historisch_2015_med_transactieprijsm2_appartementen_PC_12345": 1816.5631,
                "Koop_historisch_2014_med_transactieprijsm2_appartementen_PC_12345": 1700,
                "Koop_historisch_2013_med_transactieprijsm2_appartementen_PC_12345": 1739.0873,
                "Koop_historisch_2012_med_transactieprijsm2_appartementen_PC_12345": 1620,
                "Koop_historisch_2020_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
                "Koop_historisch_2018_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_vrijstaanden_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
                "Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
                "Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
                "Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
                "Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
                "Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345": -1,
                "Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345": -1
            ],
            [
                "BU_g_woz_v2018": 149,
                "BU_g_woz_v2016": 125,
                "WK_g_woz_v2019": 219,
                "BU_g_ink_po_v2017": 23.1,
                "Woonoppervlakte": 80,
                "BAGbouwjaar": 1956,
                "maanden_sinds_jan2004": 1,
                "BUURT_m2_alle_objecten": 413897,
                "Woningtype_appartement": 1,
                "BAG_perceeloppervlakte": 2465,
                "Woningtype_vrijstaand": 0,
                "Woningtype_tussenwoning": 0,
                "BU_ste_oad_v2018": 2874,
                "WK_ste_oad_v2018": 3460,
                "Hoofdweg_Distance": 4703,
                "hoogte": 25.92,
                "Gemeentehuis_Distance": 3097,
                "Trein_Distance": 1992,
                "Park_Distance": 1919,
                "WIJK_aantal_objecten": 21470,
                "Autosnelweg_Distance": 648,
                "BUURT_median_bouwjaar_appartementen": 1963,
                "Koop_historisch_2020_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2019_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2018_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2017_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2016_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2015_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2014_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2013_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2012_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2011_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2010_med_transactieprijsm2_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_PC_12345": 2946.4833,
                "Koop_historisch_2019_med_transactieprijsm2_PC_12345": 2745.1923,
                "Koop_historisch_2018_med_transactieprijsm2_PC_12345": -1,
                "Koop_historisch_2017_med_transactieprijsm2_PC_12345": 2120,
                "Koop_historisch_2016_med_transactieprijsm2_PC_12345": -1,
                "Koop_historisch_2015_med_transactieprijsm2_PC_12345": 1486.6667,
                "Koop_historisch_2014_med_transactieprijsm2_PC_12345": 1433.3333,
                "Koop_historisch_2013_med_transactieprijsm2_PC_12345": 1325,
                "Koop_historisch_2012_med_transactieprijsm2_PC_12345": -1,
                "Koop_historisch_2011_med_transactieprijsm2_PC_12345": -1,
                "Koop_historisch_2010_med_transactieprijsm2_PC_12345": -1,
                "Koop_historisch_2020_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2019_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2018_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2017_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2016_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2015_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2014_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2013_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2012_med_transactieprijsm2_appartementen_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_appartementen_PC_12345": 2945.2055,
                "Koop_historisch_2019_med_transactieprijsm2_appartementen_PC_12345": 2745.1923,
                "Koop_historisch_2018_med_transactieprijsm2_appartementen_PC_12345": -1,
                "Koop_historisch_2017_med_transactieprijsm2_appartementen_PC_12345": 2120,
                "Koop_historisch_2016_med_transactieprijsm2_appartementen_PC_12345": -1,
                "Koop_historisch_2015_med_transactieprijsm2_appartementen_PC_12345": 1486.6667,
                "Koop_historisch_2014_med_transactieprijsm2_appartementen_PC_12345": 1433.3333,
                "Koop_historisch_2013_med_transactieprijsm2_appartementen_PC_12345": 1318.75,
                "Koop_historisch_2012_med_transactieprijsm2_appartementen_PC_12345": -1,
                "Koop_historisch_2020_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
                "Koop_historisch_2018_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_vrijstaanden_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
                "Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
                "Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
                "Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
                "Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
                "Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456": -1,
                "Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345": -1,
                "Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345": -1,
                "Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345": -1]
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

In [27]:
print("End of notebook")

End of notebook
